In [1]:
import torch
from torch import nn

In [2]:
class FEBAM(nn.Module):
    def __init__(self, x_dim, y_dim):
        super(FEBAM, self).__init__()
        self.W = nn.Linear(x_dim, y_dim)
        self.sigmoid = nn.Sigmoid()
        self.V = nn.Linear(y_dim, x_dim)

    def forward(self, x):
        update_w = self.W.weight + (y0 - y1
        y0 = self.W(x)
        y0 = self.sigmoid(y0)
        x1 = self.V(y0)
        x1 = self.sigmoid(x1)
        y1 = self.W(x1)
        y1 = self.sigmoid(y1)
        return x1, y1

In [3]:
model = FEBAM(10, 10)

In [ ]:
w = w + lr * (y0 - yt) @ (x0 + xt).T